#listings/latest 호출


In [ ]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import pandas as pd
import json


url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'3301',
  'limit':'3220',
  'convert':'USD',
}

#API_KEY는 자신의 API키로 바꿔줘야 한다.
headers = {
  'Accepts': 'application/json',
#  'X-CMC_PRO_API_KEY': '7bf50af0-6214-40eb-8d63-90831fc465da',
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  datas = json.loads(response.text)
  print(datas)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)




{'status': {'timestamp': '2021-09-10T10:50:54.872Z', 'error_code': 0, 'error_message': None, 'elapsed': 307, 'credit_count': 17, 'notice': None, 'total_count': 6522}, 'data': [{'id': 5490, 'name': 'Lux Bio Cell', 'symbol': 'LBXC', 'slug': 'lux-bio-cell', 'num_market_pairs': 6, 'date_added': '2020-04-21T00:00:00.000Z', 'tags': [], 'max_supply': None, 'circulating_supply': 0, 'total_supply': 9674560934, 'platform': {'id': 1027, 'name': 'Ethereum', 'symbol': 'ETH', 'slug': 'ethereum', 'token_address': '0xffe510a92434a0df346c5e72a3494b043cf249eb'}, 'cmc_rank': 3301, 'last_updated': '2021-09-10T10:50:05.000Z', 'quote': {'USD': {'price': 0.00040309172354, 'volume_24h': 275358.51480884, 'percent_change_1h': -0.04249799, 'percent_change_24h': -0.57158785, 'percent_change_7d': -6.3954832, 'percent_change_30d': -28.62257199, 'percent_change_60d': -43.76230062, 'percent_change_90d': -7.04061967, 'market_cap': 0, 'market_cap_dominance': 0, 'fully_diluted_market_cap': 3899735.44, 'last_updated': '2

#Info

In [ ]:
def info_id(id):
  url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/info'
  parameters = {
    #'slug':slug,
    #'limit':'249',
    #'convert':'USD'
    #'aux' : 'urls,description,logo'
    'id':id
  }

#API_KEY는 자신의 API키로 바꿔줘야 한다.
  headers = {
    'Accepts': 'application/json',
    'X-CMC_PRO_API_KEY': '7bf50af0-6214-40eb-8d63-90831fc465da',
  }

  session = Session()
  session.headers.update(headers)

  try:
    time.sleep(2)
    response = session.get(url, params = parameters)
    data = json.loads(response.text)
#    print(data)
    return data
  except (ConnectionError, Timeout, TooManyRedirects) as e:
    print(e)


def list_chunk(lst, n):
    return [lst[i:i+n] for i in range(0, len(lst), n)]


In [ ]:
import time

def get_info(datas):
  id_list_all=[]        #6500개의 id를 일차원 list로 저장
  id_list_div=[]   #quote쿼리를 위해서 100개씩 id를 잘라서 저장
  info={}
  for i in range(len(datas['data'])):
    id = datas['data'][i]['id']
    id_list_all.append(id)

  id_input_div = list_chunk(id_list_all,100) #100개단위로 리스트 저장

  for i in range(len(id_input_div)): #100개씩 쿼리를 진행
    id=''
    data=[]
    #id_list_100 = str(id_input_div[i])
    for j in range(len(id_input_div[i])):
      id += str(id_input_div[i][j]) + ','

    if i == 0 :
      info =  info_id(id[0:-1])
    else :
      info_tmp = info_id(id[0:-1])
      for f in info_tmp['data'].keys():
        info['data'][f] = info_tmp['data'][f]

  return info

  


#합치기

In [ ]:
def datas_info_gather(datas,info):
  for i in range(len(datas['data'])):
    id = str(datas['data'][i]['id'])
    datas['data'][i]['logo'] = info['data'][id]['logo']
    datas['data'][i]['description'] = info['data'][id]['description']
    datas['data'][i]['subreddit'] = info['data'][id]['subreddit']
    datas['data'][i]['urls'] = info['data'][id]['urls']
    datas['data'][i]['twitter_username'] = info['data'][id]['twitter_username']
  
  return datas

In [ ]:
info = get_info(datas)    #datas 에는 249개의 원래 데이터들 / info에는 추가적인 정보들이 있다.

In [ ]:
datas = datas_info_gather(datas,info)

# CSV파일로 내보내기

In [ ]:
import pandas as pd

df = pd.json_normalize(datas['data'])
df.to_csv("./sample3.csv")